In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.utils import shuffle, resample
import tokenizing
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.utils import to_categorical
from sklearn.exceptions import DataConversionWarning
import warnings
from scipy import sparse

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
DATADIR = os.getenv('DATADIR')

In [3]:
DATADIR = "/Users/felisialoukou/Documents/taxonomy_data"

In [4]:
new_content = pd.read_csv(
    os.path.join(DATADIR, 'new_content.csv.gz'),
    dtype=object,
    compression='gzip')

In [5]:
new_content.shape

(83200, 26)

In [6]:
new_content.columns

Index(['base_path', 'body', 'combined_text', 'content_id',
       'content_purpose_document_supertype', 'content_purpose_subgroup',
       'content_purpose_supergroup', 'description', 'details', 'document_type',
       'document_type_gp', 'email_document_supertype', 'first_published_at',
       'government_document_supertype', 'locale',
       'navigation_document_supertype', 'primary_publishing_organisation',
       'public_updated_at', 'publishing_app',
       'search_user_need_document_supertype', 'taxon_id', 'taxons', 'title',
       'untagged_type', 'updated_at', 'user_journey_document_supertype'],
      dtype='object')

In [7]:
new_content.head()

,base_path,body,combined_text,content_id,content_purpose_document_supertype,content_purpose_subgroup,content_purpose_supergroup,description,details,document_type,...,primary_publishing_organisation,public_updated_at,publishing_app,search_user_need_document_supertype,taxon_id,taxons,title,untagged_type,updated_at,user_journey_document_supertype
0,/government/publications/safety-lessons-moorin...,this summary concerns a fatal accident when a ...,safety lessons: assessing ship mooring arrange...,20c12b35-d139-44a6-a5ea-2e46836c5dcf,NaN,NaN,NaN,a safety message to the shipping industry on t...,{'emphasised_organisations': ['9c66b9a3-1e6a-4...,research,...,Marine Accident Investigation Branch,2008-03-31T00:00:00.000+00:00,whitehall,government,NaN,NaN,safety lessons: assessing ship mooring arrange...,untagged,2017-12-19 11:16:20.914,thing
1,/government/publications/school-teachers-revie...,this document contains the following informati...,school teachers' review body fifteenth report ...,5ef94d7c-7631-11e4-a3cb-005056011aef,NaN,NaN,NaN,this document contains the following informati...,{'emphasised_organisations': ['ab9a4afd-4e0f-4...,research,...,School Teachers' Review Body,2005-12-05T00:00:00.000+00:00,whitehall,government,NaN,NaN,school teachers' review body fifteenth report ...,untagged,2017-12-20 15:23:01.588,thing
2,/government/publications/solubility-studies-in...,superplasticisers are chemical admixtures used...,solubility studies in the presence of polycarb...,b136eb16-4ace-4bbe-9ff3-40fee997fd73,reports,NaN,NaN,experimental studies to gain greater insight i...,{'emphasised_organisations': ['bd692b3d-2d4c-4...,research,...,Nuclear Decommissioning Authority,2015-08-14T08:00:00.000+00:00,whitehall,government,NaN,NaN,solubility studies in the presence of polycarb...,untagged,2018-02-13 11:49:31.811,thing
3,/government/publications/vat-rate-change-compl...,research report looking at the costs and impac...,vat rate change: compliance costs and impact o...,5fe826ca-7631-11e4-a3cb-005056011aef,reports,NaN,NaN,research on the compliance costs and commercia...,{'emphasised_organisations': ['6667cce2-e809-4...,research,...,HM Revenue & Customs,2010-10-11T08:08:00.000+00:00,whitehall,government,NaN,NaN,vat rate change: compliance costs and impact o...,untagged,2018-01-03 16:28:37.530,thing
4,/government/publications/reliability-of-vocati...,this project investigated the reliability of t...,reliability of vocational assessment: level 3 ...,60559837-7631-11e4-a3cb-005056011aef,reports,NaN,NaN,research into the reliability of three electro...,{'emphasised_organisations': ['83f6e93f-bb2c-4...,research,...,Ofqual,2013-01-17T09:30:00.000+00:00,whitehall,government,NaN,NaN,reliability of vocational assessment: level 3 ...,untagged,2018-01-29 11:09:48.811,thing


In [8]:
# 'base_path', u'body', u'combined_text', u'content_id',
#        u'content_purpose_document_supertype', u'content_purpose_subgroup',
#        u'content_purpose_supergroup', u'description', u'details',
#        u'document_type', u'document_type_gp', u'email_document_supertype',
#        u'first_published_at', u'government_document_supertype', u'locale',
#        u'navigation_document_supertype', u'primary_publishing_organisation',
#        u'public_updated_at', u'publishing_app',
#        u'search_user_need_document_supertype', u'taxon_id', u'taxons',
#        u'title', u'untagged_type', u'updated_at',
#        u'user_journey_document_supertype'],

In [9]:
new_content.drop(['content_purpose_document_supertype', 'content_purpose_subgroup',
                'content_purpose_supergroup','email_document_supertype',
                'government_document_supertype','navigation_document_supertype',
                'public_updated_at', 'search_user_need_document_supertype', 
                'taxon_id','taxons','user_journey_document_supertype','updated_at'],axis=1,inplace=True)

### Metadata vectorization

In [11]:
def to_cat_to_hot(meta_df, var):
    """one hot encode each metavar"""
    encoder = LabelEncoder()
    metavar_cat = var + "_cat"  # get categorical codes into new column
    meta_df[metavar_cat] = encoder.fit_transform(meta_df[var])
    tf.cast(meta_df[metavar_cat], tf.float32)
    return to_categorical(meta_df[metavar_cat])

In [12]:
def create_meta(balanced_df):

    # extract content_id index to df
    meta_df = pd.DataFrame(new_content['content_id'])
    meta_varlist = (
        'document_type',
        'first_published_at',
        'publishing_app',
        'primary_publishing_organisation'
    )

    for meta_var in meta_varlist:
        meta_df[meta_var] = meta_df['content_id'].map(
            dict(zip(new_content['content_id'], new_content[meta_var]))
        )

    # convert nans to empty strings for labelencoder types
    meta_df = meta_df.replace(np.nan, '', regex=True)

    dict_of_onehot_encodings = {}
    for metavar in meta_varlist:
        if metavar != "first_published_at":
            print(metavar)
            dict_of_onehot_encodings[metavar] = to_cat_to_hot(meta_df, metavar)

    # First_published_at:
    # Convert to timestamp, then scale between 0 and 1 so same weight as binary vars
    meta_df['first_published_at'] = pd.to_datetime(meta_df['first_published_at'])
    first_published = np.array(
        meta_df['first_published_at']
    ).reshape(
        meta_df['first_published_at'].shape[0],
        1
    )

    scaler = MinMaxScaler()
    first_published_scaled = scaler.fit_transform(first_published)

    last_year = np.where(
        (
            (np.datetime64('today', 'D') - first_published).astype('timedelta64[Y]')
            <
            np.timedelta64(1, 'Y')
        ),
        1,
        0
    )

    last_2years = np.where(
        (
            (np.datetime64('today', 'D') - first_published).astype('timedelta64[Y]')
            <
            np.timedelta64(2, 'Y')
        ),
        1,
        0
    )

    last_5years = np.where(
        (
            (np.datetime64('today', 'D') - first_published).astype('timedelta64[Y]')
            <
            np.timedelta64(5, 'Y')
        ),
        1,
        0
    )

    olderthan5 = np.where(
        (
            (np.datetime64('today', 'D') - first_published).astype('timedelta64[Y]')
            >
            np.timedelta64(5, 'Y')
        ),
        1,
        0
    )

    meta = np.concatenate(
        (dict_of_onehot_encodings['document_type'],
         dict_of_onehot_encodings['primary_publishing_organisation'],
         dict_of_onehot_encodings['publishing_app'],
         first_published_scaled,
         last_year,
         last_2years,
         last_5years,
         olderthan5),
        axis=1
    )

    return sparse.csr_matrix(meta),meta_df

In [13]:
meta,meta_df = create_meta(new_content)

document_type
publishing_app
primary_publishing_organisation


/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype datetime64[ns] was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/ipykernel_launcher.py:43: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/ipykernel_launcher.py:53: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/ipykernel_launcher.py:63: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.


In [14]:
meta_df.head()

,content_id,document_type,first_published_at,publishing_app,primary_publishing_organisation,document_type_cat,publishing_app_cat,primary_publishing_organisation_cat
0,20c12b35-d139-44a6-a5ea-2e46836c5dcf,research,2008-03-31 00:00:00,whitehall,Marine Accident Investigation Branch,57,9,184
1,5ef94d7c-7631-11e4-a3cb-005056011aef,research,2005-12-05 00:00:00,whitehall,School Teachers' Review Body,57,9,274
2,b136eb16-4ace-4bbe-9ff3-40fee997fd73,research,2015-08-14 08:00:00,whitehall,Nuclear Decommissioning Authority,57,9,221
3,5fe826ca-7631-11e4-a3cb-005056011aef,research,2010-10-11 08:08:00,whitehall,HM Revenue & Customs,57,9,142
4,60559837-7631-11e4-a3cb-005056011aef,research,2013-01-17 09:30:00,whitehall,Ofqual,57,9,248
